## $V_B^-$ coupled to three nitrogen nuclear spins via Fermi contact

In [1]:
from qutip import *
from matplotlib.pyplot import figure
from scipy.signal import argrelextrema
import numpy as np
import time
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

In [2]:
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

In [3]:
#Nuclear Spins
N = 3
spin = 1

Ix = spin_Jx(1)
Iy = spin_Jy(1)
Iz = spin_Jz(1)
iden = identity(3)

In [4]:
#Electron Spins
Sx, Sy, Sz = sigmax(), sigmay(), sigmaz()

In [5]:
# zero field splitting
Dzfs = 2*np.pi*3.471e9

# gyromagnetic ratios
gammae = 2*np.pi*2.79925e10
gammaN = 2*np.pi*3.078e6

# background magnetic field
# B0 = 0.33e-3

# # Fermi contact hyperfine coupling
# Ahf_xx = 2*np.pi*np.array([80.219, 46.110, 80.202])*1e6
# Ahf_yy = 2*np.pi*np.array([57.486, 91.571, 57.479])*1e6
# Ahf_zz = 2*np.pi*np.array([47.957, 47.935, 47.935])*1e6
# Ahf_xy = 2*np.pi*np.array([19.687, -0.004, -19.687])*1e6

# Fermi contact hyperfine coupling
Ahf_xx = 2*np.pi*np.array([ 79.406, 46.944, 79.406])*1e6
Ahf_yy = 2*np.pi*np.array([ 58.170, 90.025, 58.170])*1e6
Ahf_zz = 2*np.pi*np.array([ 48.159, 48.158, 48.159])*1e6
Ahf_xy = 2*np.pi*np.array([ 18.391, 0.000, -18.391])*1e6

# Normalization
units = 2*np.pi*1e6
De = Dzfs/units
ge = gammae/units
gN = gammaN/units

Axx = Ahf_xx/units
Ayy = Ahf_yy/units
Azz = Ahf_zz/units
Axy = Ahf_xy/units

A_perp = (Axx + Ayy)/(2*np.sqrt(2))
B_perp = (Axx - Ayy)/(2*np.sqrt(2))

AA = [Axx, Ayy, Azz, Axy, A_perp, B_perp]

In [6]:
#interaction operators
SIxx = [tensor(Sx,Ix,iden,iden),tensor(Sx,iden,Ix,iden),tensor(Sx,iden,iden,Ix)]
SIyy = [tensor(Sy,Iy,iden,iden),tensor(Sy,iden,Iy,iden),tensor(Sy,iden,iden,Iy)]
SIzz = [tensor(Sz,Iz,iden,iden),tensor(Sz,iden,Iz,iden),tensor(Sz,iden,iden,Iz)]
SIxy = [tensor(Sx,Iy,iden,iden),tensor(Sx,iden,Iy,iden),tensor(Sx,iden,iden,Iy)]
SIyx = [tensor(Sy,Ix,iden,iden),tensor(Sy,iden,Ix,iden),tensor(Sy,iden,iden,Ix)]

#local S
Szz = tensor(Sz,iden,iden,iden)
Sxx = tensor(Sx,iden,iden,iden)
Syy = tensor(Sy,iden,iden,iden)

#local I
Ixx = [tensor(identity(2),Ix,iden,iden),tensor(identity(2),iden,Ix,iden),tensor(identity(2),iden,iden,Ix)]
Iyy = [tensor(identity(2),Iy,iden,iden),tensor(identity(2),iden,Iy,iden),tensor(identity(2),iden,iden,Iy)]
Izz = [tensor(identity(2),Iz,iden,iden),tensor(identity(2),iden,Iz,iden),tensor(identity(2),iden,iden,Iz)]

In [7]:
#initial state
#electron state
ket0 = basis(2,0)
ket1 = basis(2,1)
xp = (ket0 + ket1)/np.sqrt(2)
xm = (ket0 - ket1)/np.sqrt(2)
rho_ep = xp*xp.dag()
rho_em = xm*xm.dag()

#set rubik state
ket_z = basis(3,1)
ket_x = (basis(3,0) - basis(3,2))/np.sqrt(2)
ket_y = (basis(3,0) + basis(3,2))/np.sqrt(2)

#initial nuclear
rho_n1 = ket_z*ket_z.dag()
rho_n = tensor(rho_n1,rho_n1,rho_n1) # |zzz>

#initial state
rho_0 = tensor(rho_ep,rho_n) # |+x>|zzz>

#projective measurement
Pqz_xp = tensor(xp*xp.dag(),iden,iden,iden)
Pqz_xm = tensor(xm*xm.dag(),iden,iden,iden)

In [8]:
#projection in rubik states
P_z = ket_z*ket_z.dag()
P_x = ket_x*ket_x.dag()
P_y = ket_y*ket_y.dag()

P_xy = P_x + P_y
P_zx = P_z + P_x
P_yz = P_y + P_z

X_xy = ket_x*ket_y.dag() + ket_y*ket_x.dag()
X_zx = ket_z*ket_x.dag() + ket_x*ket_z.dag()
X_yz = ket_y*ket_z.dag() + ket_z*ket_y.dag()

Y_xy = (1j)*ket_x*ket_y.dag() - (1j)*ket_y*ket_x.dag()
Y_zx = (1j)*ket_z*ket_x.dag() - (1j)*ket_x*ket_z.dag()
Y_yz = (1j)*ket_y*ket_z.dag() - (1j)*ket_z*ket_y.dag()

Z_xy = ket_x*ket_x.dag() - ket_y*ket_y.dag()
Z_zx = ket_z*ket_z.dag() - ket_x*ket_x.dag()
Z_yz = ket_y*ket_y.dag() - ket_z*ket_z.dag()

Xc_xy = tensor(X_xy,X_xy,X_xy)
Xc_zx = tensor(X_zx,X_zx,X_zx)
Xc_yz = tensor(X_yz,X_yz,X_yz)

Yc_xy = tensor(Y_xy,Y_xy,Y_xy)
Yc_zx = tensor(Y_zx,Y_zx,Y_zx)
Yc_yz = tensor(Y_yz,Y_yz,Y_yz)

Zc_xy = tensor(Z_xy,Z_xy,Z_xy)
Zc_zx = tensor(Z_zx,Z_zx,Z_zx)
Zc_yz = tensor(Z_yz,Z_yz,Z_yz)

#rotating in Z
U_z1 = tensor(P_z - P_xy,P_z - P_xy,P_z - P_xy) #pi rotation
U_z2 = tensor(P_z + (1j)*X_xy,P_z + (1j)*X_xy,P_z + (1j)*X_xy) #pi over 2 
U_z4 = tensor(P_z + (P_xy + (1j)*X_xy)/np.sqrt(2),P_z + (P_xy + (1j)*X_xy)/np.sqrt(2),P_z + (P_xy + (1j)*X_xy)/np.sqrt(2)) #pi over 4

E_z2 = (U_z2 + U_z2.dag())/2
O_z2 = (U_z2 - U_z2.dag())/2
E_z4 = (U_z4 + U_z4.dag())/2
O_z4 = (U_z4 - U_z4.dag())/2

#rotating in X
U_x1 = tensor(P_x - P_yz,P_x - P_yz,P_x - P_yz) #pi rotation
U_x2 = tensor(P_x + (1j)*X_yz,P_x + (1j)*X_yz,P_x + (1j)*X_yz) #pi over 2 
U_x4 = tensor(P_x + (P_yz + (1j)*X_yz)/np.sqrt(2),P_x + (P_yz + (1j)*X_yz)/np.sqrt(2),P_x + (P_yz + (1j)*X_yz)/np.sqrt(2)) #pi over 4

E_x2 = (U_x2 + U_x2.dag())/2
O_x2 = (U_x2 - U_x2.dag())/2
E_x4 = (U_x4 + U_x4.dag())/2
O_x4 = (U_x4 - U_x4.dag())/2

#rotating in Y
U_y1 = tensor(P_y - P_zx,P_y - P_zx,P_y - P_zx) #pi rotation
U_y2 = tensor(P_y + (1j)*Y_zx,P_y + (1j)*Y_zx,P_y + (1j)*Y_zx) #pi over 2 
U_y4 = tensor(P_y + (P_zx + (1j)*Y_zx)/np.sqrt(2),P_y + (P_zx + (1j)*Y_zx)/np.sqrt(2),P_y + (P_zx + (1j)*Y_zx)/np.sqrt(2)) #pi over 4

E_y2 = (U_z2 + U_z2.dag())/2
O_y2 = (U_z2 - U_z2.dag())/2
E_y4 = (U_z4 + U_z4.dag())/2
O_y4 = (U_z4 - U_z4.dag())/2

In [9]:
#target state: X*rho0
rho_targ = Xc_yz*rho_n*Xc_yz

In [10]:
#simple test
U_ent = tensor(rho_ep,U_x2) + tensor(rho_em,U_x2.dag()) #tensor(rho_ep,-1j*Xc_yz) + tensor(rho_em,(1j*Xc_yz).dag())
rho_ztest = (U_ent*rho_0*U_ent.dag()).unit()
fidelity(rho_targ,rho_ztest.ptrace([1,2,3]))

1.00000002186654

In [11]:
N_rev = 20 #revolution rotation, will change to N_alpha in mitigation  but do we need that in I_z? maybe no

#B for a given phi and revolution N
def Bop(phi,N):
    B = (4*N*A_perp.mean()/phi + De + 0.5*Azz.mean())/(ge-gN)
    # B = (4*N*A_perp.mean()/phi + De -0.5*Azz.mean())/(ge+gN)
    return B
    
# testing part
#B0 = 125.614e-3 # Delta ~ A
#B0 = 200e-3 # fast Delta Sigma
#B0 = 124.84038e-3 # Sigma << Delta

# off_set = 0 #N_rev/2
# B0 = Bop(np.pi/2,N_rev+off_set) #B from pi/4, given N_rev to have max at t = N_rev*T_p , in Tesla

A_perp = (Axx + Ayy)/(2*np.sqrt(2))
B_perp = (Axx - Ayy)/(2*np.sqrt(2))
# w0 = ge*B0 - De
# wn = gN*B0
# w = np.array([wn,wn,wn]) - 0.5*Azz
# Delta = np.array([w0,w0,w0]) - w
# Sigma = np.array([w0,w0,w0]) + w
# p = 5
# T_p = p*(2*np.pi/Delta.mean())
# tau = T_p/4

In [12]:
Bop(np.pi/2,N_rev) #B from pi/4, in Tesla

np.float64(0.21324750484450467)

In [13]:
def Fx(t,T_p):
    tau = T_p/4
    t_m = (t/T_p - t//T_p)*T_p # remnent in the period  
    t_check = np.array([t_m<=3*tau,t_m>tau])
    if t_check.all():  #t_m<=3*tau and t_m>tau:
        return -1
    else: 
        return 1

def Fy(t,T_p):
    tau = T_p/4
    t_m = (t/T_p - t//T_p)*T_p # remnent in the period  
    t_check = np.array([t_m<=3*tau,t_m>tau])
    # if all(t_check):  #t_m<=3*tau and t_m>tau:
    #     return -1
    # else: 
    #     return 1
    return 1

def Fz(t,T_p):  
    tau = T_p/4
    t_m = (t/T_p - t//T_p)*T_p # remnent in the period  
    t_check = np.array([t_m<=3*tau,t_m>tau])
    if t_check.all():  #t_m<=3*tau and t_m>tau:
        return -1
    else: 
        return 1

In [14]:
#Effective Hamiltonian
def H_exact_B(t,B0,p):
    Axx = AA[0]
    Ayy = AA[1]
    Azz = AA[2]
    Axy = AA[3]

    # w0 = ge*B0 - De
    # wn = gN*B0
    # w = np.array([wn,wn,wn]) - 0.5*Azz
    # Delta = np.array([w0,w0,w0]) - w
    # Sigma = np.array([w0,w0,w0]) + w
    # T_p = p*(2*np.pi/Delta.mean())
    
    w0 = De - ge*B0 
    wn = gN*B0
    w = np.array([wn,wn,wn]) + 0.5*Azz
    Delta = np.array([w0,w0,w0]) - w
    Sigma = np.array([w0,w0,w0]) + w
    T_p = p*(2*np.pi/np.abs(Delta.mean()))
    
    Hz = 0
    for k in range(3):
        Hz += Fz(t,T_p)*Azz[k]*SIzz[k]/2

    #rotating terms
    H_Delta = 0
    for k in range(3):
        H_Delta += A_perp[k]*np.cos(Delta[k]*t)*(Fx(t,T_p)*SIxx[k] + Fy(t,T_p)*SIyy[k]) #A_perp[k]*(SIxx[k]) 
        H_Delta -= A_perp[k]*np.sin(Delta[k]*t)*(Fx(t,T_p)*SIxy[k] - Fy(t,T_p)*SIyx[k])

    #counter rotating terms
    H_Sigma = 0
    for k in range(3):
        H_Sigma += B_perp[k]*np.cos(Sigma[k]*t)*(Fx(t,T_p)*SIxx[k] - Fy(t,T_p)*SIyy[k])
        H_Sigma += B_perp[k]*np.sin(Sigma[k]*t)*(Fy(t,T_p)*SIyx[k] + Fx(t,T_p)*SIxy[k])
        H_Sigma -= Axy[k]*np.sin(Sigma[k]*t)*(Fx(t,T_p)*SIxx[k] - Fy(t,T_p)*SIyy[k])/(2*np.sqrt(2))
        H_Sigma += Axy[k]*np.cos(Sigma[k]*t)*(Fx(t,T_p)*SIxy[k] + Fy(t,T_p)*SIyx[k])/(2*np.sqrt(2))

    #control style
    H_C = 0
    # for k in range(3):
    #     H_C += fy(t)*tensor(Sy,iden,iden,iden)
    
    H = Hz + H_Delta + H_Sigma + H_C

    return H

In [15]:
# T2 decoherence
# Gamma_2 = np.array([2*np.pi/(8e-7),2*np.pi/(2e-6),2*np.pi/(3e-6),2*np.pi/(4e-6),2*np.pi/(8e-6)]) # at room temperature WRONG
Gamma_2 = np.array([1/(2e-6), 1/(4e-6)]) # at room temperature
Gamma_2 = Gamma_2/units

In [16]:
N_rev = [50] #[10,20,30,40,50] #[60,70,80,90,100]
Blist = [Bop(np.pi/2,N) for N in N_rev]

p = 5
T_p_max = p*(2*np.pi/((ge*np.max(Blist) - De) - gN*np.max(Blist) - 0.5*Azz.mean()))
M = 500
fold = 1
tlist = np.linspace(0, fold*np.max(N_rev)*T_p_max, M+1) 

#fidelity
fidel_exact = np.zeros(shape=[len(Gamma_2),len(tlist)])

cP_yz = tensor(qeye(2),P_yz,P_yz,P_yz)
oper_targ = 1j*tensor(sigmax(),Xc_yz)


def task(k):
    fidel_temp = np.zeros(shape=[len(tlist)])
    B0 = Blist[0]
    def H_exact(t):
        return H_exact_B(t,B0,p)
        
    # prop = propagator(QobjEvo(H_z),tlist)
    prop_exact = propagator(QobjEvo(H_exact),tlist,c_ops=[np.sqrt(Gamma_2[k])*Szz])
    
    for j in range(len(tlist)):
        prop_exact[j] = prop_exact[j]*to_super(cP_yz)
    
    for j in range(len(tlist)):
        # fidel[k,j] = average_gate_fidelity(prop[j], oper_targ)/average_gate_fidelity(oper_targ, oper_targ)
        fidel_temp[j] = average_gate_fidelity(prop_exact[j], oper_targ)/average_gate_fidelity(oper_targ, oper_targ)

    return fidel_temp

results = loky_pmap(task,range(len(Gamma_2)),progress_bar=True,map_kw={"num_cpus":16,"fail_fast":True})

for k in range(len(Gamma_2)):
    for j in range(len(tlist)):
        fidel_exact[k,j] = results[k][j]

50.0%. Run time: 10696.02s. Est. time left: 00:02:58:16
100.0%. Run time: 10994.12s. Est. time left: 00:00:00:00
Total run time: 11000.64s


In [17]:
import pickle
pickle.dump(fidel_exact, open("Fidelity_Weyl_X-zy-axis-Szz", "wb"))